A partir del dataset elegido y ya preprocesado con la lógica definida en el TP1, el grupo debe realizar el entrenamiento y evaluación de al menos 3 algoritmos de machine learning:

- Se debe elegir y definir una métrica de performance a utilizar para evaluar los modelos. Fundamentar la elección de la métrica.

- Se debe aplicar alguna técnica de feature engineering para mejorar la data de entrada a los modelos, y mostrar la comparativa de los resultados obtenidos en cada caso. Si no es posible o útil, fundamentar el motivo por el cual no se realizará

- Por cada modelo, se debe entrenarlo y evaluar su comportamiento con la métrica elegida. Sacar conclusiones respecto a los motivos por los cuales el modelo funciona bien o mal.

- Se deben utilizar técnicas que garanticen que los modelos no están sobreentrenando sin que nos demos cuenta.


- Determinar el valor final de la métrica que podría ser informado al cliente, utilizando técnicas que permitan obtener un valor lo más realista posible. Fundamentar.